# imports & settings

In [9]:
import numpy as np
import pandas as pd
%precision %.3f
from os.path import join
from glob import glob
from tqdm import tqdm
import ntpath 
from copy import deepcopy
import pytz
TZ = pytz.FixedOffset(540) # GMT+09:00; Asia/Seoulsilent

import sys  
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
import utils
from utils import *
import recept_dataset
from feature_preprocessing import impute_support_features, impute, normalization

from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn import metrics 
import warnings
from functools import reduce
from itertools import product

import matplotlib
matplotlib.rc('font', size=22)
from matplotlib import pyplot as plt

RANDOM_STATE=utils.RANDOM_STATE
DATAROOT = utils.DATAROOT
BALANCED_TRAINING = True
ALREADY_EXTRACTED_SUBFEATURES = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Function definitions for classification experiment

### metrics

In [2]:
def perf_metrics(y_true: np.ndarray, y_pred: np.ndarray):
    y_pred_cls = np.rint(y_pred)    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore") 
        acc, acc_bal = (metrics.accuracy_score(y_true, y_pred_cls), 
            metrics.balanced_accuracy_score(y_true, y_pred_cls))
        f1_score = metrics.f1_score(
            y_true=y_true, y_pred=y_pred_cls,average='binary',pos_label=1)
        auc = metrics.roc_auc_score(
            y_true=y_true, y_score=y_pred, average='macro'
        )
    return dict(
        ACC=acc*100, ACC_BAL=acc_bal*100, F1_score = f1_score*100, AUC=auc
    )

    
def group_inner_split(X_train,y_train, pids):
    inner_splitter = GroupKFold(n_splits=5)
    for dev_index, val_index in inner_splitter.split(X_train, y_train, groups = pids):
        return dev_index, val_index

def stratified_inner_split(X_train,y_train):
    inner_splitter = StratifiedKFold(n_splits=5)
    for dev_index, val_index in inner_splitter.split(X_train, y_train):
        return dev_index, val_index

    
    
def get_feature_names(df):
    return df.columns[df.columns.str.contains('#')].to_list()


### classifiers

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb
import catboost


classifiers = {
    'lr':LogisticRegression(random_state = RANDOM_STATE, max_iter=500 ),
    'lr_lasso_C=0.1':LogisticRegression(
        penalty='l1', solver='saga' ,random_state = RANDOM_STATE
        , max_iter=500, C=0.1
    ),
    'lr_lasso_C=0.01':LogisticRegression(
        penalty='l1', solver='saga' ,random_state = RANDOM_STATE
        , max_iter=500, C=0.01
    ),
    'lr_lasso_C=0.1_maxiter=1000':LogisticRegression(
        penalty='l1', solver='saga'
        ,random_state = RANDOM_STATE, max_iter=1000
        , C=0.1
    ),
    'knn':KNeighborsClassifier(),
    'svm':SVC(probability=True),
    'gp':GaussianProcessClassifier(),
    'dt':DecisionTreeClassifier(random_state = RANDOM_STATE),
    'rf':RandomForestClassifier(random_state = RANDOM_STATE),
    'mlp':MLPClassifier(),
    'adaboost':AdaBoostClassifier(),
    'gnb':GaussianNB(),
    'qda':QuadraticDiscriminantAnalysis(),
    'catboost': catboost.CatBoostClassifier(
        random_seed=RANDOM_STATE, eval_metric='AUC'
    )    
    #,'xgboost':xgb.Booster()
    }



classifier_names = {
    'lr':'LogisticRegression'
    ,'lr_lasso_C=0.1':'lr_lasso_C=0.1'
    ,'lr_lasso_C=0.01':'lr_lasso_C=0.01'
    ,'lr_lasso_C=0.1_maxiter=1000':'lr_lasso_C=0.1_maxiter=1000'
    
    ,'knn':'KNeighborsClassifier',
    'svm':'SVM',
    'gp':'GaussianProcess',
    'dt':'DecisionTree',
    'rf':'RandomForest',
    'mlp':'Multi-layer Perceptron',
    'adaboost':'AdaBoost',
    'gnb':'Gaussian Naive Bayes',
    'qda':'QuadraticDiscriminantAnalysis',
    'catboost':'CatBoost'
    #,'xgboost':'XGBoost'
}

### run_classification

In [4]:


def run_classification(df, use_ray = True,  cat_features=None
    , classifier_name='catboost',
    feature_selection = None, experiment_name=''
) :                

    X = df[get_feature_names(df)]    
    y = df['receptivity'].replace({'receptive':1,'non-receptive':0}) 
    pids = df['pid']

    results = LOGO_5fold(
        X,y, pids, feature_selection=feature_selection,
        cat_features=cat_features,
        classifier_name=classifier_name
        ,use_ray=use_ray
    )
    
    results.insert(
        results.shape[1], 'experiment',experiment_name     
    )
    return results
    


### run trial

In [5]:
from sklearn.linear_model import  Lasso
def run_trial(X, y,pids,  train_index,test_index, verbose,            
              feature_selection = None, cat_features=None, 
              classifier_name='catboost'):

    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    if BALANCED_TRAINING:
        from imblearn.over_sampling import RandomOverSampler
        ros = RandomOverSampler(random_state=RANDOM_STATE)
        X_train, y_train = ros.fit_resample(X_train, y_train)
    
    if feature_selection['status']:
        if feature_selection['method']=='LASSO':
            cv = StratifiedKFold(shuffle= True, random_state=RANDOM_STATE)
    
            classifier = Lasso(tol=1e-3)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                classifier.fit(X=X_train, y=y_train)
            coef=np.abs(classifier.coef_)
            I = coef.argsort()
            keep_ratio=feature_selection['keep_ratio']
            num_features_to_keep = int(keep_ratio*X_train.shape[1])            

            X_train =  X_train[X_train.columns[I[-1*num_features_to_keep:]]]
            
    feature_names =X_train.columns.tolist()
    if verbose:  
        print("X_train.shape",X_train.shape)

    clf = classifiers[classifier_name]

    if classifier_name=='catboost':
        d_train = catboost.Pool(
            data = X_train,
            label = y_train,
            feature_names = feature_names,
            cat_features = cat_features
        )
        
        clf.fit(X = d_train,          
           verbose_eval=False,
           early_stopping_rounds=20,
                #plot=True
        )
    else:
        clf.fit(X_train, y_train)
        
    prob = clf.predict_proba(X.iloc[test_index][feature_names])[:,1]
    test_metrics = perf_metrics(y.iloc[test_index], prob)    

    prob = clf.predict_proba(X.iloc[train_index][feature_names])[:,1]
    train_metrics = perf_metrics(y.iloc[train_index], prob)

    test_pid = pids.iloc[test_index].unique()[0]    
    
    res = {'test_pid':test_pid}
    res.update({f'TEST_{k}':v  for k,v in test_metrics.items()})
    res.update({f'TRAIN_{k}':v  for k,v in train_metrics.items()})
    return res    

### CV

In [6]:


def LOGO_5fold(
    X,y, pids, cat_features=None, classifier_name='catboost',
    feature_selection = None
    , use_ray=False
):
    
    splitter =  GroupKFold(
        n_splits=5
    )
    func = ray.remote(run_trial).remote if use_ray else run_trial

    results = []    
    for i, (train_index, test_index) in tqdm(
        enumerate(splitter.split(X, y, groups=pids))):        
        results.append(func(
                X,y, pids, train_index,test_index,
                cat_features=cat_features, classifier_name=classifier_name,
                feature_selection = feature_selection,verbose=False
        )) 
    results = ray.get(results) if use_ray else results
    results = pd.DataFrame(results).set_index('test_pid')    
    results.insert(results.shape[1],'CV_TYPE','LOGO_5fold')
    return results


# Data Split

In [7]:

LABEL_DTYPES = {
            'valence':float
            ,'arousal': float
            ,'attention': float
            ,'stress': float
            ,'duration': float
            ,'change': float
            , 'pid':str            
        }
labels = pd.read_csv(
    join(DATAROOT,'binned_esm_data.csv'), 
    dtype=LABEL_DTYPES,
    parse_dates=['timestamp']
).set_index(['pid','timestamp'])

pids = labels.index.get_level_values('pid').unique().tolist()
N = len(pids)
r = .5 # rule mining pariticpants ratio
arm_pids = pids[:int(N*r)]

labels_arm = labels[labels.index.get_level_values('pid').isin(arm_pids)]
labels_ml = labels[~labels.index.get_level_values('pid').isin(arm_pids)]
labels_arm.shape, labels_ml.shape

((1580, 20), (1754, 20))

In [8]:
labels_arm.index.get_level_values('pid').nunique(), labels_ml.index.get_level_values('pid').nunique()

(36, 37)

# RuleGenerateSet


In [9]:
labels_arm.receptivity.value_counts()

receptive        913
non-receptive    667
Name: receptivity, dtype: int64

In [10]:
labels_ml.receptivity.value_counts()

receptive        1025
non-receptive     729
Name: receptivity, dtype: int64

In [11]:

print(f'num of interventions: {labels_arm.shape[0]}')
num_participants = labels_arm.index.get_level_values('pid').nunique()


sub_winsize = 20 # min
print('number of sub-windows (transactions) for the given feature:\n num_participants*days_of_collection*collection_hours*MIN_IN_HOUR/sub_winsize')
num_participants*utils.COLLECTION_DAYS*utils.COLLECTION_HOURS*utils.MIN_IN_HOUR//sub_winsize

num of interventions: 1580
number of sub-windows (transactions) for the given feature:
 num_participants*days_of_collection*collection_hours*MIN_IN_HOUR/sub_winsize


9072

### Select window size
- which window size
    - 40  MIN
    - 80  MIN
    

- how many sub window size
    - 2
    - 4
    - 8

In [11]:
windows = [40,80,160]
num_sub_windows = [2,4,8]


In [12]:

with on_ray(object_store_memory=2e10, ignore_reinit_error=True, num_cpus=40):    
    for nsub, window_size_in_min in product(num_sub_windows,windows):
        print(f'{window_size_in_min}MIN at {nsub} subwindows')
        sub_features = recept_dataset.parallellize_extract_sub(
            labels=labels_arm
            , w_size_in_min = window_size_in_min
            ,num_sub=nsub
        )
        break


2022-08-28 03:21:19,918	INFO worker.py:1518 -- Started a local Ray instance.


40MIN at 2 subwindows


  0%|          | 0/36 [00:00<?, ?it/s](extract_sub pid=29925) /datadrive/receptivity/analysis/recept_dataset.py:72: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=29925)   _skew = sp.skew(_x, bias=False)
(extract_sub pid=29925) /datadrive/receptivity/analysis/recept_dataset.py:76: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=29925)   _kurt = sp.kurtosis(_x, bias=False)
(extract_sub pid=29925) /datadrive/receptivity/analysis/recept_dataset.py:72: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=29925)   _skew = sp.skew(_x, bias=False)
(extract_sub pid=29925) /datadrive/re

KeyboardInterrupt: 

#### Extract sub features

In [14]:
from itertools import product

with on_ray(object_store_memory=2e10, ignore_reinit_error=True, num_cpus=40):    
    for nsub, window_size_in_min in product(num_sub_windows,windows):
        print(f'{window_size_in_min}MIN at {nsub} subwindows')
        sub_features = recept_dataset.parallellize_extract_sub(
            labels=labels_arm
            , w_size_in_min = window_size_in_min
            ,num_sub=nsub
        )
        sub_features.to_csv(
            f'feature/arm/subfeature_{window_size_in_min}MIN_{nsub}.csv')

2022-08-27 09:20:09,399	INFO worker.py:1518 -- Started a local Ray instance.


40MIN at 2 subwindows


  0%|          | 0/36 [00:00<?, ?it/s](extract_sub pid=100865) /mnt/juma/receptivity/analysis/recept_dataset.py:72: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=100865)   _skew = sp.skew(_x, bias=False)
(extract_sub pid=100865) /mnt/juma/receptivity/analysis/recept_dataset.py:75: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=100865)   _kurt = sp.kurtosis(_x, bias=False)
(extract_sub pid=100865) /mnt/juma/receptivity/analysis/recept_dataset.py:72: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=100865)   _skew = sp.skew(_x, bias=False)
(extract_sub pid=100865) /mnt/juma

80MIN at 2 subwindows


  0%|          | 0/36 [00:00<?, ?it/s](extract_sub pid=100809) /mnt/juma/receptivity/analysis/recept_dataset.py:72: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=100809)   _skew = sp.skew(_x, bias=False)
(extract_sub pid=100809) /mnt/juma/receptivity/analysis/recept_dataset.py:75: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=100809)   _kurt = sp.kurtosis(_x, bias=False)
(extract_sub pid=100809) /mnt/juma/receptivity/analysis/recept_dataset.py:72: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
(extract_sub pid=100809)   _skew = sp.skew(_x, bias=False)
(extract_sub pid=100809) /mnt/juma

#### find missing features per combination

In [ ]:
measurement = pd.DataFrame( columns=[
    'window size','number of subwindows','percentage of missing features'
])
with on_ray(object_store_memory=2e10, ignore_reinit_error=True, num_cpus=40):    
    for i, (window_name, window_size_in_min) in enumerate(windows.items()):
        for _ ,nsub in enumerate(num_sub_windows):
        
            sub_features = pd.read_csv(
                f'feature/arm/subfeature_{window_size_in_min}MIN_{nsub}.csv'
            ).set_index(['pid','timestamp','sub_timestamp']) #sub_features_d[(nsub,window_name)]
            sub_features = impute_support_features(sub_features)
            ds = sub_features.isnull().sum(axis=0)  
            ds = ds.sort_values(ascending=True).values/sub_features.shape[1]
            ds = ds/(len(sub_features))
            
            measurement = pd.concat(
                [
                    measurement
                    ,pd.DataFrame({
                        'window size':[window_name]
                        ,'number of subwindows':[nsub]
                        ,'percentage of missing         features':[ds.sum()*100]
                    })
                ]
                , ignore_index=True
            )

pd.set_option("display.precision", 2)
measurement.to_csv(f'setting/missing_feature_for_each_combinations.csv'
                   , index=False)
measurement

In [ ]:
sub_features = pd.read_csv(
                f'feature/arm/subfeature_{80}MIN_{4}.csv'
            ).set_index(['pid','timestamp','sub_timestamp'])
sub_features

In [ ]:
sub_features = pd.read_csv(
                f'feature/arm/subfeature_{160}MIN_{4}.csv'
            ).set_index(['pid','timestamp','sub_timestamp'])
sub_features


### aggregated features 
#### compute

In [ ]:
EXTRACTED_AGGRAGETED_FEATURES = False
EXTRACTED_SUBFEATURES = False
def extract_sub_features(window_size_in_min=80, nsub=4):
    sub_features = dataset.parallellize_extract_sub(
        labels=labels_arm, 
        w_size_in_min= window_size_in_min
        ,num_sub=nsub
    )
    sub_features.to_csv(
        f'feature/arm/subfeature_{window_size_in_min}MIN_{nsub}.csv')

def read_sub_features(window_size_in_min=80, nsub=4):
    if not EXTRACTED_SUBFEATURES:
        extract_sub_features(window_size_in_min, nsub)
    sub_features = pd.read_csv(
        f'feature/arm/subfeature_{window_size_in_min}MIN_{nsub}.csv'
        ,dtype={'pid':str}
        ,parse_dates=['timestamp','sub_timestamp']
    ).set_index(['pid','timestamp','sub_timestamp'])
    return sub_features


def extract_aggregated_features(window_size_in_min=80, nsub=4):
    sub_features = read_sub_features(window_size_in_min, nsub)
    sub_features_preprocessed = impute_support_features(
            normalization(sub_features)
    )

    agg_feature = sub_features_preprocessed.groupby(
        ['pid','timestamp']
    ).agg(['mean','std'])
    agg_feature.columns = agg_feature.columns.map('|'.join).str.strip('|')
    agg_feature.to_csv(
        f'feature/arm/agg_feature_{window_size_in_min}MIN_{nsub}.csv'
    )

In [ ]:
import dataset 
window_size_in_min, nsub = 80, 4 # selected above

if not EXTRACTED_AGGRAGETED_FEATURES:
    extract_aggregated_features()
    
agg_feature_new = pd.read_csv(
    f'feature/arm/agg_feature_{window_size_in_min}MIN_{nsub}.csv'
    ,dtype={'pid':str}
    ,parse_dates=['timestamp']
).set_index(['pid','timestamp'])
agg_feature_new

In [ ]:
import dataset 
window_size_in_min, nsub = 80, 4 # selected above

if not EXTRACTED_AGGRAGETED_FEATURES:
    extract_aggregated_features()
    
agg_feature = pd.read_csv(
    f'feature/arm/agg_feature_{window_size_in_min}MIN_{nsub}.csv'
    ,dtype={'pid':str}
    ,parse_dates=['timestamp']
).set_index(['pid','timestamp'])
agg_feature

In [ ]:
agg_feature.equals(agg_feature_new)

In [ ]:
agg_feature.isnull().sum(axis=0).sort_values(ascending=False).head(20)

#### Feature selection

##### missing count thresholding

In [ ]:
ds_missing = agg_feature.isnull().sum(axis=0)/len(agg_feature)
#thresholding
agg_feature_percent_missing  = agg_feature.loc[:,(ds_missing[ds_missing<.2]).index]
num_eliminated = agg_feature.shape[1]-agg_feature_percent_missing.shape[1]
num_total = agg_feature.shape[1]
print(f"{num_eliminated}/{num_total} features were missing more than 20\% of the time")

plt.figure(figsize=(10,5))
N, D = agg_feature.shape
ds_missing.sort_values(ascending=False).plot(linewidth=3)
I = np.arange(0,D,50)
plt.xticks(I,I, fontsize=12);
plt.grid()
plt.xlabel('Sorted Feature Index')
plt.ylabel('Percentage of missing cases')


##### varience thresholding

In [ ]:
STD_cutoff_threshold = agg_feature_percent_missing.std().sort_values().iloc[280]

agg_feature_good_variance = agg_feature_percent_missing.iloc[
    :,agg_feature_percent_missing.std().values>STD_cutoff_threshold
]
agg_feature_good_variance.shape[1],'features kept ',agg_feature_percent_missing.shape[1]

In [ ]:

plt.figure(figsize=(10,5))
agg_feature_percent_missing.std().sort_values().plot(rot=60, linewidth=3)

plt.grid()
plt.ylabel('STD of the given feature')
plt.xlabel('Sorted Feature index')

I = np.arange(0,agg_feature_percent_missing.shape[1],40)
plt.xticks(I,I)

cutoff_threshold = 280
plt.plot([cutoff_threshold,cutoff_threshold],[0,.5],color='red', linewidth=5)
plt.text(200,0.25,'Cut off threshold')


##### Pairwise Correlation

###### prepare features order
- features highly correlated with target variable should come first in corr matrix


In [ ]:
importance_order = pd.merge(
    labels_arm[['disturbance']],
    agg_feature_percent_missing
    ,left_index=True, right_index=True
).corr()['disturbance'].abs().sort_values(ascending=False).index.to_list()[1:]

Matrix = agg_feature_percent_missing[importance_order].corr()

Matrix

In [ ]:
pd.set_option("display.precision", 2)
matplotlib.rc('font', size=22)

import seaborn as sns
sns.heatmap(
    Matrix, cmap="YlGnBu",fmt='.2f' 
    ,annot=True, mask=np.triu(np.ones_like(Matrix))
)

In [ ]:
pd.set_option("display.precision", 2)
Matrix.style.background_gradient(cmap='coolwarm')

###### selection with pairwise correlation
- find features to be eliminate
- store kept features

In [ ]:
PAIRWISE_CORR_THRESHOLD = 0.8
eliminated_features = []
for r in range(Matrix.shape[0]):
    featurename = Matrix.index[r]   
    if str(featurename) in eliminated_features:
        continue
    pairwise_corr = Matrix.iloc[r,:r]
    eliminated_features+= pairwise_corr[
        pairwise_corr>PAIRWISE_CORR_THRESHOLD
    ].index.to_list()

kept_features_after_pairwise_correlation = list(
    set(Matrix.index.to_list()) - set(eliminated_features)
)
len(kept_features_after_pairwise_correlation)

###### fit classifier after eliminating multi colliniearity

In [ ]:
data = pd.merge(
    labels_arm[['receptivity']],
    impute(agg_feature_percent_missing[kept_features_after_pairwise_correlation])
    ,left_index=True, right_index=True
)

In [ ]:
feature_selection={'status':False}
use_ray = False

measure = run_classification(
    data.reset_index(), 
    use_ray=use_ray, feature_selection=feature_selection,
    experiment_name='lr'
    , classifier_name='lr'
)
measure.groupby(['CV_TYPE','experiment']).mean()

In [ ]:
feature_selection={'status':False}
use_ray = False

measure = run_classification(
    data.reset_index(), 
    use_ray=use_ray, feature_selection=feature_selection,
    experiment_name='lr'
    , classifier_name='lr'
)
measure.groupby(['CV_TYPE','experiment']).mean()

##### LogReg+Lasso 

In [ ]:

from random import random


clf = LogisticRegression(penalty='l1', solver='saga', C=1, random_state=RANDOM_STATE)
clf.fit(
    data.drop(['receptivity'], axis=1)
    , data['receptivity']
)


In [ ]:
matplotlib.rc('font', size=22)

plt.figure(figsize=(10,7))
plt.grid()
plt.plot(np.sort(np.abs(clf.coef_[0])), linewidth=5, 
    label = 'LogReg+Lasso Coefficients'
)
plt.xlabel('Sorted Feature Index')
plt.ylabel("Absolute value of coefficient")

plt.plot(
    [161,161],[0,3], color='red', linewidth=3
    , label='Zero coefficient threshold'
)
plt.legend()


In [ ]:
lasso_features = np.array(kept_features_after_pairwise_correlation)[np.abs(clf.coef_[0])>0]
len(lasso_features)

###### test fit

In [ ]:
feature_selection={'status':False}
use_ray = False

measure_catboost = run_classification(
    pd.merge(
        labels_arm[['receptivity']],
        impute(agg_feature_percent_missing[
            #kept_features_after_pairwise_correlation
            lasso_features
        ])
        ,left_index=True, right_index=True
    ).reset_index(), 
    use_ray=use_ray, feature_selection=feature_selection,
    experiment_name='lr-after logreg-lasso'
    , classifier_name='lr'
)
measure_catboost.groupby(['CV_TYPE','experiment']).mean()

In [ ]:
feature_selection={'status':False}
use_ray = False

measure_catboost = run_classification(
    pd.merge(
        labels_arm[['receptivity']],
        impute(agg_feature_good_variance[
            lasso_features
        ])
        ,left_index=True, right_index=True
    ).reset_index(), 
    use_ray=use_ray, feature_selection=feature_selection,
    experiment_name='rf-after logreg-lasso'
    , classifier_name='rf'
)
measure_catboost.groupby(['CV_TYPE','experiment']).mean()

#### end of feature selection

In [ ]:
len(lasso_features)

In [ ]:
selected_features = lasso_features
pd.DataFrame({
    'features1':selected_features[:40]
    ,'features2':selected_features[40:40+40]
    ,'features3':selected_features[40+40:40+40+40]
}).to_csv('selected_features.csv',index=False)

In [ ]:
selected_features

### Mine Association Rules
- on RuleGenerateSet
- select sub featurs using RuleGenerate Set
- 

#### Feature selection
- extract sub features for the selected window size
- compute aggregated features
- exclude multicolliniear features
    - select aggregated features with correlation threshold of .06 

## map selected aggregated features to subfeatures

In [ ]:
selected_features_mapped = list(
    set(map(lambda x: x[:x.find('|')], selected_features))
)
selected_features_mapped = list(
    map(lambda x: x.replace('#80MIN#20MIN',''), selected_features_mapped)
)
print('sub features selected (mapped from agg. features):'
      ,len(selected_features_mapped))

#### extract sliding features


In [ ]:
ALREADY_EXTRACTED_SLIDINGFEATURES = False

def prepare_sliding_features():
    import dataset
    with on_ray(object_store_memory=2e10, ignore_reinit_error=True):    
        sliding_features = dataset.parallellize_extract_sliding(
                labels=labels_arm 
                , _sw_size_in_min = 20
                , selected_features=selected_features_mapped 
            )
    sliding_features = impute_support_features(sliding_features)
    sliding_features.to_csv('feature/arm/sliding_features.csv')

if not ALREADY_EXTRACTED_SLIDINGFEATURES:
    prepare_sliding_features()
sliding_features = pd.read_csv(
    'feature/arm/sliding_features.csv'
    , dtype={'pid':'str'}
    , parse_dates=['timestamp']
).set_index(['pid','timestamp'])


sliding_features.isnull().sum().sort_values(ascending=False)     


In [ ]:
plt.figure(figsize=(10,5))

(sliding_features.isnull().sum(axis=0).sort_values(ascending=False)/len(sliding_features)).plot(
    linewidth=3)
plt.grid(True)
plt.xticks(rotation=45, fontsize=10)

In [ ]:
imputed_sliding_features = impute(sliding_features)

#### Recode

In [ ]:

    
from copy import deepcopy        
def discretize_df(data, cols, pid):
    df = deepcopy(data)
    count2 = 0
    count1 = 0
    count3 = 0
    for col in cols:
        try:
            df[col] = pd.qcut(df[col], 3, labels=["l","m","h"])
            count3 += 1
        except:
            try:
                df[col] = pd.qcut(df[col], 2, labels=["l","h"])
                count2 += 1
            except:
                df[col] = pd.qcut(df[col], 1, labels=["m"])
                count1 += 1
        print(col, df[col].values_counts())
    df.insert(0,'pid',pid)
    df = df.reset_index().set_index(['pid']+data.index.names)
    return df


df_recoded = [] 
for pid in tqdm(imputed_sliding_features.index.get_level_values('pid').unique()):    
    res = discretize_df(imputed_sliding_features.loc[pid],
                        imputed_sliding_features.columns.tolist(), pid)
    df_recoded.append(res)
df_recoded = pd.concat(df_recoded)
df_recoded = df_recoded.astype('str').fillna('missing').astype('category')
df_recoded.head()

### convert to transaction list

In [ ]:
dataset = []
for i, r in df_recoded.iterrows():
    transaction = [f'{featurename}:{value}' 
                   for featurename, value in zip(r.index, r.values)]
    dataset.append(transaction)
len(dataset)
    

#### mine Rules

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import (
    apriori, fpmax,fpgrowth, association_rules
)


# dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
#            ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
#            ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
#            ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
#            ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df


In [ ]:
frequent_itemsets = fpgrowth(
    df, min_support=0.6, use_colnames=True,
    max_len=5
)
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets

In [ ]:
res = association_rules(frequent_itemsets,  min_threshold=0.1)
res

In [ ]:
res=res.assign(
    consequent_length = res['consequents'].apply(lambda x: len(x))
    ,antecedant_length = res['antecedents'].apply(lambda x: len(x))
    , lift_log_scale_abs = np.log(res.lift)
)

# only keep the rules with consequent_length = 1
res = res[res.consequent_length==1]
res

In [ ]:
topk_rules = pd.concat(
    [res.sort_values(['lift_log_scale_abs'],ascending=False)[:50]
    , res.sort_values(['lift_log_scale_abs'],ascending=False)[-50:]]
)
    
topk_rules

In [ ]:
topk_rules['antecedents'] = topk_rules['antecedents'].apply(lambda x: set(x))
topk_rules['consequents'] = topk_rules['consequents'].apply(lambda x: set(x))
topk_rules.to_csv('top100_rules.csv', index=False)

In [ ]:
topk_rules_df = pd.read_csv('top100_rules.csv')
topk_rules_df

# MLTrainEvalSet

In [ ]:
num_interventions= labels_ml.shape[0]
print('there are {:.0f} participants with {} interventions in total'.format( labels_ml.index.get_level_values('pid').nunique(),num_interventions))

In [ ]:
test_size = .2
dev_interventions = (1-test_size)*num_interventions
print("Dev:Test = {:.0f}:{:.0f} ".format( dev_interventions, num_interventions-dev_interventions))


val_size = .2
print("Train:Val = {:.0f}:{:.0f} ".format( (1-val_size)*dev_interventions, test_size*dev_interventions))

### Full features

In [ ]:

with on_ray(object_store_memory=2e10, ignore_reinit_error=True, num_cpus=38):    
    full_featurs_ml = dataset.parallellize_extract(
        labels=labels_ml, 
        w_name=window_name,
        w_size = window_size,
        use_ray=True            
    )

        
    

#### Select the features

In [ ]:
selected_features_mapped_to_fullFeatures = list(map(lambda x: x.replace('30MIN#',''), selected_features_mapped))
full_features_ml = full_features_ml[selected_features_mapped_to_fullFeatures]

#### imputation

In [ ]:
full_features_ml.isnull().sum()

In [ ]:

full_features_ml = impute_by_filling_support_features(full_features_ml)
full_features_ml.iloc[:,full_features_ml.columns.str.contains('SUP')].std().sum()

In [ ]:
df_sub_ml = impute_with_mean_of_participant(df_sub_ml)
df_sub_ml.isnull().sum()

#### Classification

In [ ]:

data = pd.merge(labels_ml[['is_opportune_moment']], full_features_ml, left_index=True, right_index=True )

measure = run_classification(data.reset_index())
measure.groupby('CV_TYPE').mean()

- using reduced ML set

In [ ]:


data = pd.merge(labels_ml[['is_opportune_moment']], full_features_ml, left_index=True, right_index=True )
data = pd.merge(data, featuresCFF[[]],left_index=True, right_index=True)# to reduce
measure = run_classification(data.reset_index())
measure.groupby('CV_TYPE').mean()


### Agg 

#### extract sub features
- extract sub features only for selected 


In [ ]:
selected_features_mapped[:3]

In [ ]:
ml_sub_features_d={}
windows = {
    '6HR': 6*60 * 60,
}

with on_ray(object_store_memory=2e10, ignore_reinit_error=True, num_cpus=35):    
    for window_name, window_size  in windows.items():
        ml_sub_features_d[window_name] = dataset.parallellize_extract_sub(
            labels=labels_ml, 
            w_name=window_name,
            w_size = window_size,
            selected_features=selected_features_mapped,
            use_ray=True            
        )

        
    

In [ ]:
df_sub_ml = ml_sub_features_d['6HR']
df_sub_ml.to_csv(f'features/ml-sub-window_6HR-numSub_12_FS.csv')
df_sub_ml = pd.read_csv(f'features/ml-sub-window_6HR-numSub_12_FS.csv', dtype={'pid':'str'},
                     parse_dates=['timestamp'])
df_sub_ml.set_index(['pid','timestamp','sub_timestamp'], inplace=True)

df_sub_ml

#### impute

In [ ]:

df_sub_ml = impute_by_filling_support_features(df_sub_ml)
df_sub_ml.iloc[:,df_sub_ml.columns.str.contains('SUP')].std().sum()

In [ ]:
df_sub_ml = impute_with_mean_of_participant(df_sub_ml)
df_sub_ml.shape

In [ ]:
agg_feature_ml = df_sub_ml.groupby(['pid','timestamp']).agg(['mean','std'])
agg_feature_ml.columns = agg_feature_ml.columns.map('|'.join).str.strip('|')
agg_feature_ml = drop_zero_varience_features(agg_feature_ml)

In [ ]:
agg_feature_ml

#### Classification performance

In [ ]:

data = pd.merge(labels_ml[['is_opportune_moment']], agg_feature_ml, left_index=True, right_index=True )

measure = run_classification(data.reset_index())
measure.groupby('CV_TYPE').mean()

- reduced set

In [ ]:
data = pd.merge(labels_ml[['is_opportune_moment']], agg_feature_ml, left_index=True, right_index=True )
data = pd.merge(data, featuresCFF[[]],left_index=True, right_index=True)# to reduce
measure = run_classification(data.reset_index())
measure.groupby('CV_TYPE').mean()

### CFF
- impute
- recode

In [ ]:
df_sub_ml = pd.read_csv(f'features/ml-sub-window_6HR-numSub_12_FS.csv', dtype={'pid':'str'},
                     parse_dates=['timestamp'])
df_sub_ml.set_index(['pid','timestamp','sub_timestamp'], inplace=True)


df_sub_ml = impute_by_filling_support_features(df_sub_ml)

df_sub_ml.isnull().sum(axis=0).sort_values(ascending=False)

#### recode

In [ ]:
df_sub_ml_recoded = [] 
for pid in tqdm(df_sub_ml.index.get_level_values('pid').unique()):    
    res = discretize_df(df_sub_ml.loc[pid],df_sub_ml.columns.tolist(), pid)
    
    df_sub_ml_recoded.append(res)
    
df_sub_ml_recoded = pd.concat(df_sub_ml_recoded)
df_sub_ml_recoded = df_sub_ml_recoded.reset_index().set_index(['pid', 'timestamp','sub_timestamp'])
df_sub_ml_recoded = df_sub_ml_recoded.astype('object').fillna('missing').astype('category')

In [ ]:
df_sub_ml_recoded.shape

#### extract 

- for each rule
  - select the df that satisfy the context
     - extract Y from contextually filtered subfeatures and add to features


In [ ]:
TOPK_RULES = 40
rules_df = pd.read_csv('rules/rules1000-mlen_10.csv', usecols=['rules','lift'], dtype={'lift':float})
rules_df = rules_df.iloc[:TOPK_RULES]
rules_df = rules_df[rules_df.lift>1.05]

#rules_df.rules = rules_df.rules.str.replace('.SUP.','.SUP:').str.replace('.','#')
featuresCFF = pd.DataFrame()# we store CFF here


for i,rule in (rules_df.iterrows()):
    context, y = rule['rules'].split(' => ')
    
    #clean `{`
    context = context[1:-1]    
    y = y[1:-1]
    
    #'BND_DST_PAC#3HR#30MIN#VAR=m' =>'BND_DST_PAC#3HR#30MIN#VAR'
    y=y.split('=')[0]    
    
    # flag the indices that satisfy our context
    index_flag = True
    for c in context.split(','): #'BND_DST_PAC#3HR#30MIN#VAR=m', 'BND_DST_SPD#3HR#30MIN#BEP=m'
        name, val = c.split('=')
        index_flag = index_flag & (df_sub_ml_recoded[name]==val)        
    featuresCFF[f'mean-{y}-context_{context}'] = df_sub_ml[index_flag][y].groupby(['pid','timestamp']).mean()
    featuresCFF[f'std-{y}-context_{context}'] = df_sub_ml[index_flag][y].groupby(['pid','timestamp']).std()
    

In [ ]:
featuresCFF.shape # topK =400 rules

In [ ]:
features.head()

In [ ]:
featuresCFF = impute_with_mean_of_participant(featuresCFF)


#### Classifier Performance

In [ ]:
data = pd.merge(labels_ml[['is_opportune_moment']], featuresCFF, left_index=True, right_index=True )
data

In [ ]:

data = pd.merge(labels_ml[['is_opportune_moment']], features, left_index=True, right_index=True )

measure = run_classification(data.reset_index())
measure.groupby('CV_TYPE').mean()

### Combining

In [ ]:
full_features_ml.shape

In [ ]:
featuresCFF.shape

In [ ]:
agg_feature_ml.shape

In [ ]:
data = pd.merge(full_features_ml, featuresCFF, left_index=True, right_index=True )
data = pd.merge(data, agg_feature_ml, left_index=True, right_index=True )
data = pd.merge(labels_ml[['is_opportune_moment']], data, left_index=True, right_index=True )
data

In [ ]:
measure = run_classification(data.reset_index())
measure.groupby('CV_TYPE').mean()